In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/My Drive")

In [3]:
import torch
from torchtext import data
import spacy
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("new_data.csv",index_col=0)

In [5]:
train_df, test_df = train_test_split(df,test_size = 0.2, random_state=42,shuffle = True)
train_df, valid_df = train_test_split(train_df,test_size=0.05, random_state=42,shuffle = True)

# BERT

In [28]:
!pip install transformers

     |████████████████████████████████| 1.4MB 7.2MB/s 
     |████████████████████████████████| 890kB 22.8MB/s 
     |████████████████████████████████| 2.9MB 14.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=231e86ca85f3b58f597c8a057c4ca9c84f978fb77404330146531190b704e094
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [29]:
import torch
import random
import numpy as np
from transformers import BertTokenizer
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [30]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens


from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [31]:
fields = [('text',TEXT), ('label',LABEL)]

train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = "/content/drive/My Drive",
                                        train = 'train.csv',
                                        validation = "valid.csv",
                                        test = 'test.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True)
BATCH_SIZE = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort_key = lambda x: len(x.text))

In [37]:
from transformers import BertTokenizer, BertModel
import torch.nn as nn

bert = BertModel.from_pretrained('bert-base-uncased')

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [48]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    prediction_list = []
    true_label= []
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            rounded_preds = torch.round(torch.sigmoid(predictions))
            prediction_list.append(rounded_preds.tolist())
            true_label.append(batch.label.tolist())
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator),np.array([int(i) for batch in prediction_list for i in batch]), np.array([int(i) for batch in true_label for i in batch])

In [38]:
LABEL.build_vocab(train_data)
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.2

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [39]:
N_EPOCHS = 8

best_valid_loss = float('inf')
loss = []
acc = []
for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc,_,_ = evaluate(model, valid_iterator, criterion)
    loss.append(train_loss)
    acc.append(train_acc)    
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 53s
	Train Loss: 0.548 | Train Acc: 71.22%
	 Val. Loss: 0.456 |  Val. Acc: 79.32%
Epoch: 02 | Epoch Time: 1m 53s
	Train Loss: 0.314 | Train Acc: 86.88%
	 Val. Loss: 0.328 |  Val. Acc: 85.54%
Epoch: 03 | Epoch Time: 1m 53s
	Train Loss: 0.250 | Train Acc: 89.85%
	 Val. Loss: 0.296 |  Val. Acc: 89.68%
Epoch: 04 | Epoch Time: 1m 53s
	Train Loss: 0.220 | Train Acc: 91.46%
	 Val. Loss: 0.250 |  Val. Acc: 90.72%
Epoch: 05 | Epoch Time: 1m 53s
	Train Loss: 0.189 | Train Acc: 92.45%
	 Val. Loss: 0.295 |  Val. Acc: 90.20%
Epoch: 06 | Epoch Time: 1m 53s
	Train Loss: 0.197 | Train Acc: 92.39%
	 Val. Loss: 0.315 |  Val. Acc: 90.72%
Epoch: 07 | Epoch Time: 1m 53s
	Train Loss: 0.148 | Train Acc: 94.36%
	 Val. Loss: 0.227 |  Val. Acc: 91.24%
Epoch: 08 | Epoch Time: 1m 53s
	Train Loss: 0.135 | Train Acc: 94.69%
	 Val. Loss: 0.287 |  Val. Acc: 91.24%


In [40]:
np.save("BERT_train_acc", acc)
np.save("BERT_train_loss", loss)

In [41]:
loss,acc, y_pred_BERT, true_label_BERT = evaluate(model, test_iterator,criterion)
print("Wth BERT, accuracy on the test set is", acc)

Wth BERT, accuracy on the test set is 0.8982460151116053


In [43]:
np.save("y_pred_BERT", y_pred_BERT)
np.save("true_label_BERT", true_label_BERT)